# Find a good number of compartments and predur

In [ ]:
import importlib

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', 'pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils

In [ ]:
import compartments_and_predur_utils as utils

# CBC5o

In [ ]:
data_folder = os.path.join('..', 'step0_preprocess_iGluSnFR_data', 'data_preprocessed')
stimulus    = pd.read_csv(os.path.join(data_folder, 'Franke2017_stimulus_time_and_amp_corrected.csv'))

stimulus.plot(x='Time', figsize=(12,3));

In [ ]:
params_default = data_utils.load_var(os.path.join('cell_params', 'CBC5o_cell_params_default.pkl'))
params_unit = data_utils.load_var(os.path.join('cell_params', 'CBC5o_cell_params_unit.pkl'))

In [ ]:
# These functions define the number of compartments for different regions (e.g. 1,2 = Dendrites)
# c is the constant, it will be normalized with rm and ri such that changing these parameters does not have an
# effect on the compartments.
# Note that, if compartment become too small, you will also need a smaller time step or the simulation becomes unstable.
# It might eventually crash or at least seem to be very noisy.

final_cpl_dict = {
    'cpl_1': {"c": 0.4,  'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.15, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0,  'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.15, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.15, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.15, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 0.4,  'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 0.3,  'rm': 'bp_rm', 'ri': 'bp_ri'},
}

params_default.update(final_cpl_dict)

## Create cell

In [ ]:
import retsim_cells

# Create cell.
cell = retsim_cells.CBC(
    bp_type = 'CBC5o',
    predur=1.0, t_rng=(1.5,2.5),
    params_default=params_default, params_unit=params_unit,
    stimulus=stimulus, stim_type='Light',
    cone_densfile       = 'dens_cone_optimized_submission2.n',
    bp_densfile         = 'dens_CBC5o_optimize_ON.n',
    nval_file           = 'nval_optimize_CBCs.n',
    chanparams_file     = 'chanparams_CBC5o_optimize_ON.n',
    expt_file_list      = ['test_ON'],
    expt_base_file_list = ['retsim_files/expt_CBC_base.cc'],
    retsim_path=os.path.abspath(os.path.join('..', 'neuronc', 'models', 'retsim')) + '/'
)

In [ ]:
cones_connect_to_nodes = [1077, 980, 1190]

# Create c++ file.
cell.create_retsim_expt_file(verbose=False, on2cone_nodes=cones_connect_to_nodes)
# Compile c++ file.
!(cd {cell.retsim_path} && make)

In [ ]:
cell.init_retsim(verbose=True)

In [ ]:
utils.set_and_plot_cpl(cell, final_cpl_dict)

## Check how fast resting Vm is reached.

In [ ]:
utils.set_and_plot_cpl(
    cell,
    { 
        'cpl_1': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
        'cpl_2': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
        'cpl_3': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
        'cpl_4': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
        'cpl_5': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
        'cpl_6': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
        'cpl_7': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
        'cpl_8': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    }
)

In [ ]:
# Set stimulus to background stimulus.
cell.stim[0]['Stim'] *= 0
plt.plot(cell.stim[0]['Stim'])
plt.show()

In [ ]:
cell.rec_type='heatmap_vm'
cell.predur = 0.0
stimdur = 10

cell.update_t_rng(t_rng=(0, 0+stimdur))
nopredur_rec_data, nopredur_rec_time, nopredur_rec_stim, tdur = utils.run_and_show_cell(cell, plot=True, verbose=True)
print(tdur)

In [ ]:
for col in nopredur_rec_data.columns:
    if 'Vm' in col:
        utils.plot_reaching_eq(
            nopredur_rec_data, nopredur_rec_time,
            nopredur_rec_stim, plt_Vm_name=col,
            Vm_eq=nopredur_rec_data[col].iloc[-1]
        )

### Small predur

In [ ]:
cell.rec_type='heatmap_vm'
cell.predur = 1.0
stimdur = 10

cell.update_t_rng(t_rng=(0, 0+stimdur))
smallpredur_rec_data, smallpredur_rec_time, smallpredur_rec_stim, tdur = utils.run_and_show_cell(cell, plot=True)
print(tdur)

In [ ]:
for col in smallpredur_rec_data.columns:
    if 'Vm' in col:
        utils.plot_reaching_eq(
            smallpredur_rec_data, smallpredur_rec_time,
            smallpredur_rec_stim, plt_Vm_name=col,
            Vm_eq=nopredur_rec_data[col].iloc[-1]
        )

### Different predurs

In [ ]:
predur_list = [0.1, 0.5, 1, 3, 4, 5, 6, 10]

predur_tdur_list = []
predur_rec_data_list = []
cell.update_t_rng(t_rng=(0, 0.1))

for idx, predur in enumerate(predur_list):
    cell.predur = predur
    rec_data, rec_time, rec_stim, tdur = utils.run_and_show_cell(cell, plot=idx==0)
    print(predur, end='\t')
    
    # Save data.
    predur_tdur_list.append(tdur)
    predur_rec_data_list.append(rec_data)

In [ ]:
importlib.reload(utils)

In [ ]:
utils.plot_params_vs_time_to_Vm_eq(
    params=predur_list, rec_data_list=predur_rec_data_list,
    Vm_eq=nopredur_rec_data['Vm (4)'].iloc[-1], plt_Vm_name='Vm (4)'
)

In [ ]:
utils.plot_params_vs_tdur(params=predur_list, tdur_list=predur_tdur_list)
print(f'{predur_tdur_list[-1] / predur_list[-1]:.2f} seconds per predur seconds')

### Different stim times

In [ ]:
stimdur_list = [0.1, 0.5, 1, 3, 6]

stimdur_tdur_list = []
stimdur_rec_data_list = []

cell.predur = 0.0

for idx, stimdur in enumerate(stimdur_list):   
    cell.update_t_rng(t_rng=(0, stimdur))
    rec_data, rec_time, rec_stim, tdur = utils.run_and_show_cell(cell, plot=idx==0)
    
    print(stimdur, end='\t')
    
    # Save data.
    stimdur_tdur_list.append(tdur)
    stimdur_rec_data_list.append(rec_data)

In [ ]:
utils.plot_params_vs_tdur(params=stimdur_list, tdur_list=stimdur_tdur_list)
print(f'{stimdur_tdur_list[-1] / stimdur_list[-1]:.2f} seconds per sim seconds')

### Different compartments

In [ ]:
utils.set_and_plot_cpl(cell, final_cpl_dict)

In [ ]:
cell.stim[0]['Stim'] *= 0
plt.plot(cell.stim[0]['Stim'])
plt.show()

In [ ]:
cell.rec_type = 'heatmap_vm'
cell.predur = 6.0
cell.predur_dt = 1e-4
cell.sim_dt = 1e-5
stimdur = 1.0

cell.update_t_rng(t_rng=(0, 0+stimdur))
nopredur_rec_data2, nopredur_rec_time2, nopredur_rec_stim2, tdur = utils.run_and_show_cell(cell, plot=True, verbose=True)
print(f'min = {tdur/60}')

In [ ]:
count = 0
for col in list(nopredur_rec_data2.columns):
    if 'Vm' in col:
        utils.plot_reaching_eq(nopredur_rec_data2, nopredur_rec_time2, nopredur_rec_stim2, plt_Vm_name=col)
        count += 1
        if count > 5: break

## Run light step

In [ ]:
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.05] = 0.3
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.1] = 1
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.15] = 0.0
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.2] = 0.8
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.25] = 0.5
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.3] = 0.0
cell.stim[0]['Stim'][cell.stim[0]['Time']>0.35] = 1.0

plt.plot(cell.stim[0]['Time'], cell.stim[0]['Stim'])
plt.xlim(0, 0.4)
plt.show()

In [ ]:
utils.set_and_plot_cpl(cell, final_cpl_dict)

In [ ]:
cell.rec_type='optimize'
cell.update_t_rng((0, 0.4))
cell.predur = 6.0
cell.predur_dt = 1e-4
cell.sim_dt = 1e-5

In [ ]:
fc_rec_data, fc_rec_time, fc_stim, tdur = utils.run_and_show_cell(cell, plot=False)
print(f'min = {tdur/60}')

if fc_rec_data is None:
    print('Was None!')
    cell.run(verbose=True, plot=True)

In [ ]:
MSEs = utils.compare_to_final_cpl(fc_rec_time, fc_rec_data, fc_rec_data)
MSEs

### Test different optimize_cpls

In [ ]:
cpl2MSE = []

In [ ]:
cpl2MSE.append(utils.run_and_compute_cpl_MSE(cell, {
    'cpl_1': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 2.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 2.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 2.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 2.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 2.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}, fc_rec_data))

In [ ]:
cpl2MSE.append(utils.run_and_compute_cpl_MSE(cell, {
    'cpl_1': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}, fc_rec_data))

In [ ]:
cpl2MSE.append(utils.run_and_compute_cpl_MSE(cell, {
    'cpl_1': {"c": 3.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 3.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 3.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}, fc_rec_data))

In [ ]:
cpl2MSE.append(utils.run_and_compute_cpl_MSE(cell, {
    'cpl_1': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 3.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 3.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}, fc_rec_data))

In [ ]:
cpl2MSE.append(utils.run_and_compute_cpl_MSE(cell, {
    'cpl_1': {"c": 2.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.4, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 2.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 2.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}, fc_rec_data))

In [ ]:
cpl2MSE.append(utils.run_and_compute_cpl_MSE(cell, {
    'cpl_1': {"c": 3.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.4, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 3.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 3.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}, fc_rec_data))

In [ ]:
cpl2MSE.append(utils.run_and_compute_cpl_MSE(cell, {
    'cpl_1': {"c": 3.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.4, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 2.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 2.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}, fc_rec_data))

In [ ]:
cpl2MSE.append(utils.run_and_compute_cpl_MSE(cell, {
    'cpl_1': {"c": 1.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.3, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 1.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 1.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}, fc_rec_data))

### Analyze tdur vs MSE

In [ ]:
utils.plot_cpl2MSE(cpl2MSE)

# Save cpl dict for optimization.

In [ ]:
# Select optimization cpl, pick one that is relatively fast but with small MSE rate and small MSE Vm
optimize_cpl_dict = {
    'cpl_1': {"c": 4.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_2': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_3': {"c": 0.0, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_4': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_5': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_6': {"c": 0.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_7': {"c": 3.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
    'cpl_8': {"c": 3.5, 'rm': 'bp_rm', 'ri': 'bp_ri'},
}

utils.set_and_plot_cpl(cell, optimize_cpl_dict)

In [ ]:
# Save dicts for optimization.
data_utils.make_dir("cell_params")
data_utils.save_var(optimize_cpl_dict, os.path.join("cell_params", cell.bp_type+"_optimize_cpl_dict.pkl"))
data_utils.save_var(final_cpl_dict, os.path.join("cell_params", cell.bp_type+"_final_cpl_dict.pkl"))

# Test sim_dt for optimize cpl

In [ ]:
utils.set_and_plot_cpl(cell, optimize_cpl_dict)

In [ ]:
sim_dt_list = [1e-6, 2e-6, 5e-6, 1e-5, 2e-5, 5e-5]

In [ ]:
cell.predur = 10
cell.predur_dt = 1e-4

cell.update_t_rng((0,0.1))

sim_dt_rec_data_list = []
sim_dt_rec_time_list = []
sim_dt_rec_tdur_list = []

for sim_dt in sim_dt_list:
    print(sim_dt)
    
    cell.sim_dt = sim_dt
    
    sim_dt_rec_data, sim_dt_rec_time, _, tdur = utils.run_and_show_cell(cell, plot=True, verbose=True)
    assert sim_dt_rec_data is not None
    print(f'min = {tdur/60:.4f}')
    
    sim_dt_rec_data_list.append(sim_dt_rec_data)
    sim_dt_rec_time_list.append(sim_dt_rec_time)
    sim_dt_rec_tdur_list.append(tdur)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15,6))
for rec_time, rec_data, sim_dt in zip(sim_dt_rec_time_list, sim_dt_rec_data_list, sim_dt_list):
    axs[0].plot(rec_time, rec_data['rate BC'].mean(axis=1), label=sim_dt)
    axs[1].plot(rec_time, rec_data['BC Vm Soma']*1e3, label=sim_dt)
axs[1].set_ylabel('mV')
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15,6))
for rec_time, rec_data, sim_dt in zip(sim_dt_rec_time_list, sim_dt_rec_data_list, sim_dt_list):
    axs[0].plot(
        rec_time[:400],
        rec_data['rate BC'].mean(axis=1)[:400] - sim_dt_rec_data_list[0]['rate BC'].mean(axis=1)[:400],
        label=sim_dt
    )
    axs[1].plot(
        rec_time[:400],
        (rec_data['BC Vm Soma'][:400] - sim_dt_rec_data_list[0]['BC Vm Soma'][:400])*1e3,
        label=sim_dt
    )
    
axs[0].set_ylabel('rate')
axs[1].set_ylabel('mV')
    
plt.legend()
plt.show()

In [ ]:
for rec_time, rec_data, sim_dt in zip(sim_dt_rec_time_list, sim_dt_rec_data_list, sim_dt_list):
    print(sim_dt, f"{rec_data['rate BC'].mean(axis=1).iloc[0]:.3f}", f"{rec_data['BC Vm Soma'].iloc[0]:.5f}")

In [ ]:
for rec_time, rec_data, sim_dt in zip(sim_dt_rec_time_list, sim_dt_rec_data_list, sim_dt_list):
    plt.plot(rec_data['BC Vm Soma'].iloc[:50]-rec_data['BC Vm Soma'].iloc[0],label=sim_dt)
plt.legend()

In [ ]:
for rec_time, rec_data, sim_dt in zip(sim_dt_rec_time_list, sim_dt_rec_data_list, sim_dt_list):
    plt.plot(rec_data['rate BC'].iloc[:50,0]-rec_data['rate BC'].iloc[0,0],label=sim_dt)
plt.legend()